#  Using Twitter Streaming API
### We will collect real-time tweet data using Twitter's streaming API
We will pass *keywords* or *hashtags* as parameters to filter in related tweets.
This is termed as ""

#### Supporting documents
1. https://developer.twitter.com/en/docs/tutorials/stream-tweets-in-real-time
2. https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/integrate/build-a-rule

# Import Packages 

In [1]:
import tweepy
import yaml
import json
from datetime import datetime

# Read twitter Authentication Keys

In [2]:
# yaml file reader funtion
def read_yaml(file_path):
    with open(file_path, "r") as f:
        return yaml.safe_load(f)

# yaml config file path
file_path = "twitter_api_key_config.yaml"
# read from config file
api_credential = read_yaml(file_path)

# Create Twitter Authentication

In [3]:
# API authentication
auth = tweepy.OAuthHandler(api_credential["api_key"], \
                           api_credential["api_secret_token"])
auth.set_access_token(api_credential["access_token"], \
                      api_credential["access_token_secret"])
api = tweepy.API(auth, wait_on_rate_limit=True)

# Define a StreamListener Object (code walkthrough)

In [17]:
# override tweepy.StreamListener to add logic to on_status
class MyStreamListenerV2(tweepy.StreamListener):
    def __init__(self):
        super(MyStreamListenerV2, self).__init__()
        self.counter = 0
        pass
    
    def on_data(self, data):
        self.counter += 1
        print(f'Tweet no: {self.counter} {json.loads(data)["text"]}, {json.loads(data)["created_at"]}')
        print('\n\n')

In [35]:
# override tweepy.StreamListener to add logic to on_status
class MyStreamListenerV1(tweepy.StreamListener):
    def __init__(self):
        super(MyStreamListenerV1, self).__init__()
        self.counter = 0
        pass
    
    def on_status(self, status):
        self.counter += 1
        print(f'Tweet No: {self.counter}. status.text: {status.text}\n')
        if self.counter >50:
            print(status)
            return False
        
    def on_data(self, data):
        self.counter += 1
        print(f'Tweet No: {self.counter}. data: {json.loads(data)["text"]}\n')
        if self.counter >50:
            print(data)
            return False
        
#     def on_data(self, data):
#         self.counter += 1
#         print(f'Tweet No: {self.counter}. status.text: {data.text}\n')
#         if self.counter >50:
#             print(data)
#             return False

# Define a StreamListener Object (code dump)

In [39]:
# override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, listen_time=60, output_file=None):
        super(MyStreamListener, self).__init__()
        self.counter = 0
        print("Initialized Tweepy StreamListener.")
        self.start_time = datetime.now()
        self.current_time = datetime.now()
        self.listen_time = listen_time
        self.output_file = output_file
        
    def on_data(self, data):
        self.current_time = datetime.now()
        time_elapsed = (self.current_time - self.start_time).total_seconds()
        if time_elapsed < self.listen_time:
            try:
                self.counter += 1
                print(f'Data no: {self.counter}. Data: {data}\n\n\n\n')
                self.output_file.write(str(data))
            except Exception as e:
                print(f"On data Exception:{e}.")
        else:
            print(f"Stream listen time period ended. Total listen time: {self.listen_time} seconds.")
            return False

    """
    def on_status(self, status):
        self.counter += 1
        print(f'Tweet No: {self.counter}. status.text: {status.text}\n'\
              f'status: {status}\n\n\n')
    """
    """
    def on_data(self, data):
        self.counter += 1
        print(f'Data no: {self.counter}. Data: {data}\n\n\n\n')
    """
    # handling Errors
    def on_error(self, status_code):
        print(f"status_code: {status_code}")
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

# Create a Stream

In [40]:
outfile = open('temp.txt', 'w')

In [41]:
myStreamListener = MyStreamListener(listen_time=60, output_file=outfile)
myStream = tweepy.Stream(api.auth, myStreamListener)

Initialized Tweepy StreamListener.


# Start a Stream

In [42]:
keywords = ['ida']
try:
    print("Stream Filter")
    myStream.filter(track=keywords)
    print("DONE")
except Exception as e:
    print(f"error in stream filter {e}")

Stream Filter
Data no: 1. Data: {"created_at":"Thu Sep 02 18:12:17 +0000 2021","id":1433492993969901569,"id_str":"1433492993969901569","text":"RT @IamGMJohnson: A reminder that the MTA spent 260 million dollars to add 500 more cops instead of ensuring the subways didn\u2019t look like a\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":251230837,"id_str":"251230837","name":"mr.Bitches","screen_name":"WZRDkenny_","location":"*curREn%tly$uNa#waR\u00bfe*","url":null,"description":"she call me Pappy when I slip on that corduroy condom","translator_type":"none","protected":false,"verified":false,"followers_count":155,"friends_count":332,"listed_count":3,"favourites_count":15626,"statuses_count":43561,"created_at":"Sat Feb 12 18:22:31

Data no: 2. Data: {"created_at":"Thu Sep 02 18:12:17 +0000 2021","id":1433492995920338944,"id_str":"1433492995920338944","text":"RT @nathanimation: I've been at this for a long time but I've ever seen it rain INSIDE a stadium\ud83d\ude33 #USOpen\n#NYC \n#Ida https:\/\/t.co\/mNNCLltesr","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":606770615,"id_str":"606770615","name":"Kimberley","screen_name":"Kimberley_Hurst","location":"Dublin City, Ireland","url":null,"description":"friends. food. foreigner. fourth. Instagram: @kimberley_hurst","translator_type":"none","protected":false,"verified":false,"followers_count":97,"friends_count":131,"listed_count":0,"favourites_count":2933,"statuses_count":2006,"created_at":"Wed Jun 13 00:50:37 +00

Data no: 5. Data: {"created_at":"Thu Sep 02 18:12:18 +0000 2021","id":1433492997690126336,"id_str":"1433492997690126336","text":"You know democrats are relieved for hurricane Ida which took their failure in Afghanistan off the top of the News cycle.","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2329624453,"id_str":"2329624453","name":"Alan M. Goldberg","screen_name":"AlanMGoldberg","location":"Los Angeles","url":"http:\/\/www.alangoldberglaw.com","description":"California Lawyer: Appeals, Civil Trials, Family Law. Litigator, mediator, arbitrator. 31 years experience. Please visit me on Linkedin.","translator_type":"none","protected":false,"verified":false,"followers_count":81,"friends_count":36,"listed_count":1,"favourites_count":

Data no: 8. Data: {"created_at":"Thu Sep 02 18:12:18 +0000 2021","id":1433493000055832584,"id_str":"1433493000055832584","text":"RT @SenSanders: Even after the devastation caused by Hurricane Ida, my Republican colleagues will say combatting climate change is too expe\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":3349075943,"id_str":"3349075943","name":"Juan \ud83c\udde8\ud83c\uddf4","screen_name":"fckn_mango3001","location":null,"url":null,"description":"there\u2019s no line at the turtle store | secretly faker | 6.22 | he\/him | proud Colombian \u2764\ufe0f","translator_type":"none","protected":false,"verified":false,"followers_count":130,"friends_count":936,"listed_count":0,"favourites_count":31770,"statuses_count":5039,"


Data no: 14. Data: {"created_at":"Thu Sep 02 18:12:19 +0000 2021","id":1433493004313141250,"id_str":"1433493004313141250","text":"\"Mr. Bravo was pleading for help in Spanish, and neighbors were trying to reach him. But water was pouring through\u2026 https:\/\/t.co\/SoDF8Wfgdm","source":"\u003ca href=\"https:\/\/mobile.twitter.com\" rel=\"nofollow\"\u003eTwitter Web App\u003c\/a\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":18378044,"id_str":"18378044","name":"Josie Ensor","screen_name":"Josiensor","location":"New York","url":"http:\/\/www.telegraph.co.uk\/authors\/josie-ensor\/","description":"US Correspondent @Telegraph. Previously in the Middle East. Recipient of Marie Colvin Award. Associate Fellow at @ICSR_Centre","translator_type":"none","protected":false,"verified":true,"followers_count":39421,"friends_count":4559,"listed_count":990,"fav

...

Data no: 281. Data: {"created_at":"Thu Sep 02 18:13:01 +0000 2021","id":1433493177743388678,"id_str":"1433493177743388678","text":"RT @IamGMJohnson: A reminder that the MTA spent 260 million dollars to add 500 more cops instead of ensuring the subways didn\u2019t look like a\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":512749889,"id_str":"512749889","name":"Sholess Paul","screen_name":"The_Paul_Menace","location":null,"url":null,"description":"I constantly forget I have a twitter.\n\n(he\/him)","translator_type":"none","protected":false,"verified":false,"followers_count":47,"friends_count":205,"listed_count":0,"favourites_count":24338,"statuses_count":4577,"created_at":"Sat Mar 03 03:47:30 +0000 2012","utc_offset":null,"t

Data no: 282. Data: {"created_at":"Thu Sep 02 18:13:01 +0000 2021","id":1433493180054392832,"id_str":"1433493180054392832","text":"RT @ksnt63: @kiara86769608 Leggi questo bellissimo articolo di @idomini \nChe gi\u00e0 aveva fatto presente i rischi\n\n\"I governi non li porta la\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1410870161985388546,"id_str":"1410870161985388546","name":"Giovanna Confalone Asero","screen_name":"AseroGiovanna","location":null,"url":null,"description":"sono sincera in modo irreversibile","translator_type":"none","protected":false,"verified":false,"followers_count":342,"friends_count":457,"listed_count":1,"favourites_count":26853,"statuses_count":21798,"created_at":"Fri Jul 02 08:00:11 +0000 2021","

Data no: 287. Data: {"created_at":"Thu Sep 02 18:13:02 +0000 2021","id":1433493183678271494,"id_str":"1433493183678271494","text":"RT @dorsogna: Una raffineria si allaga dopo l'uragano Ida a 25 miglia da New Orleans. E' la Phillips 66 Alliance Refinery, che rilascia sci\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":961233370821914624,"id_str":"961233370821914624","name":"\u2721\ud83c\udff3\ufe0f\u200d\ud83c\udf08\ud83d\udc27Simo","screen_name":"Simo07827689","location":null,"url":null,"description":"Il tiranno trover\u00e0 sempre il modo di opprimere il giusto","translator_type":"none","protected":false,"verified":false,"followers_count":1084,"friends_count":396,"listed_count":1,"favourites_count":246580,"statuses_count":2

Data no: 290. Data: {"created_at":"Thu Sep 02 18:13:03 +0000 2021","id":1433493187486617611,"id_str":"1433493187486617611","text":"RT @headlines24_7: \ud83d\udea8NEW:  President Biden says Ida shows \u2018climate crisis\u2019 has struck  VIA @breakingnewsie","source":"\u003ca href=\"https:\/\/dlvrit.com\/\" rel=\"nofollow\"\u003edlvr.it\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1229752606714728454,"id_str":"1229752606714728454","name":"MI6 ROGUE.COM","screen_name":"mi6rogue","location":null,"url":"https:\/\/www.mi6rogue.com","description":"Exposing Corruption, Geo-Politics, Policing Updates, #COVID19 Facts.\nSubmit Leaks: mi6rogue@protonmail.com","translator_type":"none","protected":false,"verified":false,"followers_count":54888,"friends_count":2846,"listed_count":168,"favourites_count":10051,"statuses_count":44710,"created_at":"Tue

Stream listen time period ended. Total listen time: 60 seconds.
DONE
